In [2]:
import calendar
import pandas as pd
import numpy as np
import requests
import pickle
import plotly.express as px

import sys
sys.path.append("../")
import src.soporte as sp
import src.biblioteca as bb

pd.set_option('max_colwidth', None)
pd.set_option("display.max_columns", None)


In [ ]:
url = f"https://apidatos.ree.es/es/datos/{category}/{widget}?{query}"
query = f"{inicio}&{final}&geo_limit=ccaa&geo_ids={id_ccaa}"

In [ ]:
balance = ["balance-electrico"]
demanda = ["evolucion", "variacion-componentes", "variacion-componentes-movil", "ire-general", "ire-general-anual", "ire-general-anual", "ire-general-movil", "ire-industria", "ire-industria-anual", "ire-industria-movil", "ire-servicios", "ire-servicios-anual", "ire-servicios-movil", "ire-otras", "ire-otras-anual", "ire-otras-movil", "demanda-maxima-diaria", "demanda-maxima-horaria", "perdidas-transporte", "potencia-maxima-instantanea", "variacion-demanda", "potencia-maxima-instantanea-variacion", "potencia-maxima-instantanea-variacion-historico", "demanda-tiempo-real", "variacion-componentes-anual"]
generacion = ["estructura-generacion", "evolucion-renovable-no-renovable", "estructura-renovables", "estructura-generacion-emisiones-asociadas", "evolucion-estructura-generacion-emisiones-asociadas", "no-renovables-detalle-emisiones-CO2", "maxima-renovable", "potencia-instalada", "maxima-renovable-historico", "maxima-sin-emisiones-historico"]
mecados = ["componentes-precio-energia-cierre-desglose", "componentes-precio", "energia-gestionada-servicios-ajuste", "energia-restricciones", "precios-restricciones", "reserva-potencia-adicional", "banda-regulacion-secundaria", "energia-precios-regulacion-secundaria", "energia-precios-regulacion-terciaria", "energia-precios-gestion-desvios", "coste-servicios-ajuste", "volumen-energia-servicios-ajuste-variacion", "precios-mercados-tiempo-real", "energia-precios-ponderados-gestion-desvios-before", "energia-precios-ponderados-gestion-desvios", "energia-precios-ponderados-gestion-desvios-after"] 

In [ ]:
inicio = "start_date=2010-01-01T00:00"
final = "end_date=2023-03-31T23:59"
corte = "time_trunc=day"
category = 
widget =
geolimit = f"&geo_limit={region}"
geo_id = f"&geo_ids={id_ccaa}"

"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2014-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=7"
"https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=day&geo_limit=ccaa&geo_ids=7"

In [163]:
df = pd.read_csv("../data/demanda.csv")

In [165]:
df["datetime"] = pd.to_datetime(df["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df['datetime'] = df['datetime'].dt.tz_convert('Europe/Paris')

In [176]:
df["datetime"].dt.year
df_year = df[df["datetime"].dt.year == 2013]

In [179]:
fig1 = px.line(df_year, x = "datetime", y = "value", line_group = None)
fig1.show()

In [ ]:
def px_demanda(año):
    df_year = df[df["datetime"].dt.year == int(año)]
    fig1 = px.line(df_year, x = "datetime", y = "value", line_group = None)
    fig1.show()

In [56]:
df2 = pd.read_csv("../data/demanda_tiempo_real.csv")
df2.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)
df2["datetime"] = pd.to_datetime(df2["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df2['datetime'] = df2['datetime'].dt.tz_convert('Europe/Paris')

In [210]:

def px_demanda_real(año, semana_del_año):
    df_year2 = df2[df2["datetime"].dt.year == int(año)]
    df_week2 = df_year2[df_year2["datetime"].dt.isocalendar().week == int(semana_del_año)]
    fig = px.line(df_week2, x='datetime', y=["value_Demanda real", "value_Demanda programada", "value_Demanda prevista"], color_discrete_sequence=['red', 'green', "blue"])
    fig.show()

In [4]:
df3 = pd.read_csv("../data/precios_mercados.csv")
df3["datetime"] = pd.to_datetime(df3["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df3['datetime'] = df3['datetime'].dt.tz_convert('Europe/Paris')

In [5]:
df3

,value,datetime
0,20.02,2014-01-01 00:00:00+01:00
1,10.34,2014-01-01 01:00:00+01:00
2,5.35,2014-01-01 02:00:00+01:00
3,5.00,2014-01-01 03:00:00+01:00
4,0.50,2014-01-01 04:00:00+01:00
...,...,...
81033,115.17,2023-03-31 19:00:00+02:00
81034,130.72,2023-03-31 20:00:00+02:00
81035,133.53,2023-03-31 21:00:00+02:00
81036,85.18,2023-03-31 22:00:00+02:00


In [34]:
df4 = df3.copy()
df4['mes_y_año'] = df4['datetime'].dt.strftime('%m-%Y')
df_mes = (df4.groupby(by = "mes_y_año")["value"].sum()).reset_index()
df_mes["mes_y_año"] =  pd.to_datetime(df_mes['mes_y_año'], format='%m-%Y')

In [37]:
df_mes.sort_values(by= "mes_y_año", ascending = True, inplace = True)

In [39]:
fig1 = px.line(df_mes, x = "mes_y_año", y = "value", line_group = None)
fig1.show()

In [54]:
año = 2016

df_year3 = df3[df3["datetime"].dt.year == int(año)]
fig1 = px.line(df_year3, x = "datetime", y = "value", line_group = None)
fig1.show()

In [67]:
balance = pd.read_csv("../data/balance.csv")

In [100]:
balance

,datetime,value_Hidráulica,percentage_Hidráulica,value_Eólica,percentage_Eólica,value_Solar fotovoltaica,percentage_Solar fotovoltaica,value_Solar térmica,percentage_Solar térmica,value_Otras renovables,percentage_Otras renovables,value_Residuos renovables,percentage_Residuos renovables,value_Generación renovable,percentage_Generación renovable,value_Turbinación bombeo,percentage_Turbinación bombeo,value_Nuclear,percentage_Nuclear,value_Ciclo combinado,percentage_Ciclo combinado,value_Carbón,percentage_Carbón,value_Motores diésel,percentage_Motores diésel,value_Turbina de gas,percentage_Turbina de gas,value_Turbina de vapor,percentage_Turbina de vapor,value_Fuel + Gas,percentage_Fuel + Gas,value_Cogeneración,percentage_Cogeneración,value_Residuos no renovables,percentage_Residuos no renovables,value_Generación no renovable,percentage_Generación no renovable,value_Consumos en bombeo,percentage_Consumos en bombeo,value_Saldo I. internacionales,percentage_Saldo I. internacionales,value_Demanda en b.c.,percentage_Demanda en b.c.,value_Hidroeólica,percentage_Hidroeólica,año
0,2013-01-01 00:00:00+01:00,66558.820,0.265905,159736.163,0.638151,10311.537,0.041195,982.070,0.003923,11133.884,0.044480,1588.423,0.006346,250310.897,1,12477.898,0.034489,145775.227,0.402922,87755.013,0.242554,31647.913,0.087475,8505.170,0.023508,1685.342,0.004658,5622.887,0.015542,-5.768,0.000016,64941.433,0.179498,3378.721,0.009339,361783.836,1,-29008.160,0.045130,-1663.129,0.002587,581423.444,1,NaN,NaN,2013
1,2013-01-02 00:00:00+01:00,80201.734,0.250916,208335.946,0.651792,14866.830,0.046512,3334.936,0.010434,11299.790,0.035352,1596.416,0.004994,319635.652,1,10361.048,0.024424,146012.579,0.344192,91926.849,0.216697,61018.324,0.143837,9294.538,0.021910,2144.466,0.005055,6730.048,0.015865,-6.089,0.000014,93396.364,0.220161,3327.517,0.007844,424205.644,1,-10689.626,0.014116,-2717.097,0.003588,730434.573,1,NaN,NaN,2013
2,2013-01-03 00:00:00+01:00,84810.292,0.345133,120064.699,0.488600,19324.717,0.078641,8319.346,0.033855,11582.596,0.047135,1630.476,0.006635,245732.126,1,12159.583,0.023715,145876.599,0.284505,118551.906,0.231213,114453.208,0.223220,9364.002,0.018263,2064.866,0.004027,6925.733,0.013507,-6.730,0.000013,99402.398,0.193866,3932.832,0.007670,512724.397,1,-13846.270,0.017485,19591.119,0.024740,764201.372,1,NaN,NaN,2013
3,2013-01-04 00:00:00+01:00,88815.608,0.433106,74322.618,0.362431,20307.624,0.099029,8700.478,0.042428,11444.623,0.055809,1475.886,0.007197,205066.837,1,10395.211,0.019019,145682.279,0.266536,126845.546,0.232073,139215.983,0.254705,9600.965,0.017566,1932.312,0.003535,7285.795,0.013330,-6.757,0.000012,101965.316,0.186553,3646.462,0.006671,546563.112,1,-7962.318,0.010250,17234.389,0.022186,760902.020,1,NaN,NaN,2013
4,2013-01-05 00:00:00+01:00,70184.389,0.310079,114679.540,0.506661,20332.367,0.089830,8075.142,0.035676,11494.886,0.050785,1577.331,0.006969,226343.655,1,3350.234,0.007133,145309.061,0.309397,86409.570,0.183986,128957.564,0.274581,9120.599,0.019420,2050.521,0.004366,7449.490,0.015862,-6.141,0.000013,82650.149,0.175982,4348.909,0.009260,469639.956,1,-7064.770,0.009990,-4112.630,0.005816,684806.211,1,NaN,NaN,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,2023-03-27 00:00:00+02:00,78368.500,0.194862,171601.992,0.426685,122356.001,0.304235,18246.578,0.045370,9287.760,0.023094,2306.200,0.005734,402175.312,1,11948.200,0.032755,167289.200,0.458606,102635.720,0.281365,11609.500,0.031826,6176.399,0.016932,1152.439,0.003159,2293.544,0.006288,NaN,NaN,57734.180,0.158272,3938.700,0.010798,364777.882,1,-22335.000,0.024953,-105806.600,0.118207,638811.594,1,8.281,0.000021,2023
3738,2023-03-28 00:00:00+02:00,90389.700,0.246848,112051.602,0.306005,130479.430,0.356330,20572.317,0.056181,10279.410,0.028072,2342.050,0.006396,366176.133,1,18896.800,0.047934,168861.800,0.428338,118740.210,0.301199,11411

In [71]:
balance["datetime"] = pd.to_datetime(balance["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
balance['datetime'] = balance['datetime'].dt.tz_convert('Europe/Paris')
balance["año"] = balance["datetime"].dt.year
df_balance_año = (balance.groupby(by = "año")["value_Generación renovable", "value_Generación no renovable",].sum()).reset_index()


C:\Users\Lin\AppData\Local\Temp\ipykernel_7164\2060100726.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [74]:
df_balance_año.info

<bound method DataFrame.info of      año  value_Generación renovable  value_Generación no renovable
0   2013                1.097577e+08                   1.635286e+08
1   2014                1.078761e+08                   1.586356e+08
2   2015                9.408811e+07                   1.733657e+08
3   2016                1.010892e+08                   1.607465e+08
4   2017                8.461087e+07                   1.776949e+08
5   2018                1.003444e+08                   1.606375e+08
6   2019                9.791330e+07                   1.629155e+08
7   2020                1.106052e+08                   1.407936e+08
8   2021                1.214591e+08                   1.385520e+08
9   2022                1.165831e+08                   1.597322e+08
10  2023                3.601783e+07                   3.347190e+07>

In [94]:
emisiones = pd.read_csv("../data/emisiones_CO2.csv")

In [97]:
emisiones["datetime"] = pd.to_datetime(emisiones["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
emisiones['datetime'] = emisiones['datetime'].dt.tz_convert('Europe/Paris')
emisiones['mes_y_año'] = emisiones['datetime'].dt.strftime('%m-%Y')
df_mes_emisiones = (emisiones.groupby(by = "mes_y_año")["value_Carbón", "value_Motores diésel", "value_Turbina de gas", "value_Turbina de vapor", "value_Ciclo combinado", "value_Cogeneración", "value_Residuos no renovables"].sum()).reset_index()
df_mes_emisiones["mes_y_año"] =  pd.to_datetime(df_mes_emisiones['mes_y_año'], format='%m-%Y')
df_mes_emisiones.sort_values(by= "mes_y_año", ascending = True, inplace = True)

C:\Users\Lin\AppData\Local\Temp\ipykernel_7164\3246576251.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [99]:
fig = px.line(df_mes_emisiones, x='mes_y_año', y=["value_Carbón", "value_Motores diésel", "value_Turbina de gas", "value_Turbina de vapor", "value_Ciclo combinado", "value_Cogeneración", "value_Residuos no renovables"])
fig.show()

In [106]:
perdidas = pd.read_csv("../data/perdidas_transporte.csv")

In [111]:
perdidas["datetime"] = pd.to_datetime(perdidas["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
perdidas['datetime'] = perdidas['datetime'].dt.tz_convert('Europe/Paris')
perdidas["percentage"] = (perdidas["percentage"] * 100)

In [120]:
año = 2022

perdidas_year = perdidas[perdidas["datetime"].dt.year == int(año)]
fig1 = px.line(perdidas_year, x = "datetime", y = "percentage", line_group = None)
fig1.show()